In [1]:
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
# Scrape Ball by ball data from any cricinfo commentary
# IPL finals 2021
url = "https://www.espncricinfo.com/series/ipl-2021-1249214/chennai-super-kings-vs-kolkata-knight-riders-final-1254117/ball-by-ball-commentary"

In [3]:
# Read the URL
req = Request( url , headers = {'User-Agent'  : 'Mozilla/5.0'})
web_byte = urlopen(req).read()
webpage = web_byte.decode('utf-8')
soup = BeautifulSoup(webpage , 'lxml')

In [5]:
# To get to array of Match Comments - 

# section - id:pane-main
# section - id:main-container
# div - class:match-page-wrapper commentary-page-wrapper
# div - class:match-body
# div - class:comment-container card
# div - class:mb-5 comments-container-body
# div - class: match-comment

column_names = ['over', 'ball', 'bowler' , 'batsman' , 'runsOffBat' , 'isWicket', 'extras']

match_data = pd.DataFrame(columns = column_names)

level1 = soup.findAll('section' , attrs = {'id' : 'pane-main'})[0]
level2 = level1.findAll('section', attrs = {'id' : 'main-container'})[0]
level3 = level2.findAll('div', attrs = {'class' : 'match-page-wrapper commentary-page-wrapper'})[0]
level4 = level3.findAll('div', attrs = {'class' : 'match-body'})[0]
level5 = level4.findAll('div', attrs = {'class' : 'comment-container card'})[0]
level6 = level5.findAll('div', attrs = {'class' : 'mb-5 comments-container-body'})[0]
level7 = level6.findAll('div', attrs = {'class' : 'match-comment'})

num = len(level7)

for i in range(num):
    level8 = level7[i].findAll('div', attrs = {'class' : 'd-flex match-comment-padder align-items-center'})[0]
    
    level9 = level8.findAll('div', attrs = {'class' : 'col-2 col-md-1 col-lg-2 match-comment-run-col'})[0]
    overNum = level9.findAll('span', attrs = {'class' : 'match-comment-over'})[0].get_text()
    runsOffBat = level9.findAll('div', attrs = {'class' : 'match-comment-run-container'})[0].findAll('div', attrs = {'class' : 'match-comment-run'})[0].get_text()

    level9Text = level8.findAll('div', attrs = {'class' : 'col-14 col-md-15 col-lg-14'})[0]
    level10Text = level9Text.findAll('div', attrs = {'class' : 'match-comment-short-text'})[0]
    CommText = level10Text.get_text().split(',')[0]
    bowler = CommText.split('to')[0]
    batsman = CommText.split('to')[1]
    over = overNum.split('.')[0]
    ball = overNum.split('.')[1]
    
    # Handle Wickets
    if (runsOffBat == 'W'):
        isWicket = 1
        runsOffBat = 0
    else:
        isWicket = 0
      
    # Handle Special cases 
    if (runsOffBat == '•'):
        runsOffBat = 0
    
    # Handle Extras 
    if (str(runsOffBat).endswith('w') or str(runsOffBat).endswith('nb')):
        extras = runsOffBat[:1]
        runsOffBat = 0 
    else: 
        extras = 0
        
    # Lot more possible cases to handle 
        
    match_data.loc[len(match_data.index)] = [over, ball, bowler, batsman, runsOffBat, isWicket, extras]
    
#print (num)

match_data.head(10)

,over,ball,bowler,batsman,runsOffBat,isWicket,extras
0,19,6,Bravo,Ferguson,1,0,0
1,19,5,Bravo,Shivam Mavi,0,1,0
2,19,4,Bravo,Shivam Mavi,0,0,0
3,19,3,Bravo,Ferguson,1,0,0
4,19,2,Bravo,Shivam Mavi,1,0,0
5,19,1,Bravo,Shivam Mavi,0,0,0
6,18,6,Thakur,Ferguson,0,0,0
7,18,5,Thakur,Ferguson,6,0,0
8,18,4,Thakur,Ferguson,2,0,0
9,18,4,Thakur,Ferguson,0,0,1
